# 4-Laboratory-29-10-2020 L

| Credits to the authors of the exercises: Andrea Pasini, Giuseppe Attanasio, Flavio Giobergia
| Master of Science in Data Science and Engineering, Politecnico di Torino, A.A. 2020-21

# KNN design and Implementation
In this exercise, you will implement your own version of the the K-Nearest Neighbors algorithm, and you will use it to assign a Iris species (i.e. a label) to flowers whose species is unknown. The KNN algorithm is straightforward. Suppose that some measurements (i.e. records) and the irrelative species are known in advance. Then, whenever we want to label a new flower, we look at the Kmost similar points (a.k.a. neighbors) and assign a label accordingly. The simplest solution is using amajority voting scheme: if the majority of the neighborsvotesfor a label, we will go for it. This approachis naive only at first sight: the local similarity assumed by KNN happens to be roughly true. Even thoughthis reasoning does not generalize well1, the KNN provides a valid baseline for your tasks.

### Steps

1. Load the Iris dataset

In [1]:
# I import in adive the libraries that I'll using during this lab
import pandas as pd
import numpy as np

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",header=None)
df.head(2)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


2. Let’s identify a portion of our data for which we will try to guess the species. Randomly select 20% of the records and store the first four columns (i.e. the features representing each flower) into atwo-dimensional numpy array of shape N×C, you can call it X_test. For the same records, store the last column (i.e. the one with the species values) into another array,namely y_test. This is the data that will be used to test the accuracy of your KNN implementationand its correct functioning (i.e. the testing data)

3. Store the remaining 80% of the records in the same way. In this case, use the namesX_trainandy_trainfor the arrays. This is the data that your model will use as ground-truth knowledge (i.e. thetraining data)

I know the possibility to do easily with scikit learn, but since we didn't study it already, I'll do it without any particular help

In [2]:
# I put everything in one script because it's easier to 
# test the algorithm with different samples

X_test = df.sample(int(len(df)/5)) #20% means len/5
y_test = X_test[4] # pandas' columns are already numpy arrays
X_test = X_test.drop(columns=[4])
X_train = df[[0,1,2,3]].drop(X_test.index)
y_train  = df[4].drop(y_test.index)

In [3]:
X_test.head()

,0,1,2,3
64,5.6,2.9,3.6,1.3
88,5.6,3.0,4.1,1.3
90,5.5,2.6,4.4,1.2
15,5.7,4.4,1.5,0.4
28,5.2,3.4,1.4,0.2


In [4]:
y_test.head()

64    Iris-versicolor
88    Iris-versicolor
90    Iris-versicolor
15        Iris-setosa
28        Iris-setosa
Name: 4, dtype: object

In [5]:
# index used
X_test.index

Int64Index([ 64,  88,  90,  15,  28,   0,  73,  29, 103,  47, 122, 108, 138,
             44,  27,  11,  80,  58, 144, 140,  53,  18,  25,  23,  39,   8,
             13,  85, 110,  75],
           dtype='int64')

In [6]:
X_train.head()

,0,1,2,3
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4


In [7]:
y_train.head()

1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
5    Iris-setosa
Name: 4, dtype: object

4. Implement now the KNN algorithm and expose it as a Python class. Implement the fit method first. Here, you should only keep track of the main attributes that will be used by the algorithm. In this version of the algorithm, does the KNN need to store all the samples of X_train and y_train?

In [8]:
""" The general structure, I'll complete later on anothe """
class KNearestNeighbors:
    def __init__(self, k, distance_metric="euclidean"):
        self.k = k
        self.distance_metric = distance_metric
        
    def fit(self, X, y):
        """
            Store the'prior knowledge' of you model that will be used 
            to predict new labels.
            
            :param X : input data points, ndarray, shape = (R,C).
            :param y : input labels, ndarray, shape = (R,).
        """
        self.X = X
        self.y = y
    
    def predict(self, X):
        """
            Run the KNN classification on X.
            
            :param X: input data points, ndarray, shape = (N,C).
            :return: labels : ndarray, shape = (N,).
        """
        pass# TODO: implement it!

Well, I think in this case we cannot say precisely, but I think we will answer later and we'll say if we notice a possible underfitting or overfitting

5. To identify the K closest points, or neighbors, a notion of distance is required. Your implementation must support three different distance definitions. Given two n-dimensional points p= (p1, p2, . . . , pn) and q= (q1, q2, . . . , qn), the euclidean distance is defined as

<center>$ed(p,q) = \sqrt{(\sum_{i=1}^{n} (p_i - q_i)^2} $</center>

The second distance function is the cosine distance, which is defined as: 

<center>$cd(p,q) = 1 - |cs(p,q)|$</center>

where <u>cs</u> is the cosine similarity, defined as:

<center>  $ cs(p,q) = \frac {\sum_{i=1}^{n} p_iq_i}{\sqrt{(\sum_{i=1}^{n}p_i^2)}\sqrt{\sum_{i=1}^{n}q_i^2}}$ </center>

The third distance is the Manhattan distance. The distance is defined as:

<center>$md(p,q) = \sum_{i=1}^{n} | p_i - q_i | $</center>

In [9]:
import math

def euclidean(a,b):
    return np.sqrt(np.sum((a-b)**2,axis=1))

def cosine(a,b):
    cs = np.sum((a*b),axis=1)/(np.sqrt(np.sum(a**2))*np.sqrt(np.sum(b**2,axis=1)))       
    return 1 - abs(cs)

def manhattan(a,b):
    return np.sum(abs(a-b),axis=1)

6. Implement thepredict method. The function receives as input a numpy array with N rows and C columns, corresponding to N flowers. The method assigns one of the three Iris species to each row using the KNN algorithm, and returns them as a numpy array. For the actual implementation, apply the identify K neighbors using the distance specified by the parameters k and distance passed to the constructor. Then, assign the label using a majority voting scheme

7. Now let’s fit the KNN model with the X_train and y_train data. Then, try to use your KNN model to predict the species for each record in X_test and store them in a nupy array called y_pred. Check how many Iris species in the array y_pred have been guessed correctly with respect to theo nes in y_test. A prediction is correct if y_pred[i] == y_test[i]. The ratio between the number of correct guesses and the total number of guesses is known as accuracy. If all labels are assigned correctly ((y_pred == y_test).all() == True), the accuracy of the model is 100%. Instead, if none of the guessed species corresponds to the real one ((y_pred == y_test).any() == False),the accuracy is 0%.

In [10]:
class KNearestNeighbors:
    
    def __init__(self, k, distance_metric="euclidean"):
        self.k = k
        self.distance_metric = distance_metric
        
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def predict(self, X):
            
        distances_list = list()

        for index, row in X.iterrows():
            
            if self.distance_metric == "euclidean":
                distances = euclidean(row,self.X)
            
            elif self.distance_metric == "cosine":
                distances = cosine(row,self.X)
            
            elif self.distance_metric == "manhattan":
                distances = manhattan(row,self.X)
            else:
                print("** ERRORE **")
                return 

            order = np.argsort(distances)
            topk = order[order < self.k]

            top_labels = self.y[topk.index]
            dic = {'Iris-setosa':0, 'Iris-versicolor':0, 'Iris-virginica':0}

            for y in top_labels:
                dic[y] += 1

            dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])}
            distances_list.append(list(dic.keys())[2])
                
        return np.array(distances_list)        

In [11]:
# euclidean, cosine, manhattan
a = KNearestNeighbors(7,"euclidean")
a.fit(X_train,y_train)
y_pred = a.predict(X_test)

print("Accuracy: ", (np.sum(y_pred == y_test))/len(y_pred)*100,"%")

Accuracy:  70.0 %


8. (*) As a software developer, you might want to increase the functionalities of your product and publish newer versions over time. The better your code is structured and organized, the lower is theeffort to release updates.As such, extend now your KNN implementation by adding the parameterweightsto the constructor. <br /> Change your KNN implementation to accept a new weighting scheme for the labels. Ifweights="distance", weight neighbor votes by the inverse of their distance (for the distance, again, usedistance_metric). Instead, if the default is chosen (weights="uniform"), use the majority voting you already imple-mented 

I'll copy the previous one in order to specify the differences

In [12]:
"""
    distance_metric = ['euclidean','cosine','manhattan']
    weights = ['uniform','distance']
"""
class KNearestNeighbors:
    
    def __init__(self, k, distance_metric="euclidean",weights="uniform"):
        self.k = k
        self.distance_metric = distance_metric
        self.weights = weights
        
    def measure_distance(self,row,refer):
        if self.distance_metric == "euclidean":
            distances = euclidean(row,refer)
        elif self.distance_metric == "cosine":
            distances = cosine(row,refer)
        elif self.distance_metric == "manhattan":
            distances = manhattan(row,refer)
        else:
            return 
        return distances
    
    def apply_weights(self,distances):
        # initialize the dic
        dic = {elem : 0 for elem in set(self.y)}
        
        # pre-save the index of the distances
        index  = distances.index
        
        # get index of top k sorted sequenceù
        if self.weights == "uniform":
            order = np.argsort(distances)[:self.k]
        elif self.weights == "distance":
            order = np.argsort(-(1/distances))[:self.k]
        
        # get relative lables and check the occurrencies
        top_labels = self.y[index[order]] 
        for y in top_labels:
            dic[y] += 1
        dic = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])}
        
        return list(dic.keys())[len(dic)-1]
            
        
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def predict(self, X):
        distances_list = list()
        for index, row in X.iterrows():
            
            distances = self.measure_distance(row,self.X)
            prediction = self.apply_weights(distances)
            
            distances_list.append(prediction)
                
        return np.array(distances_list)   

In [13]:
test = KNearestNeighbors(5,"euclidean","distance")
test.fit(X_train,y_train)
y_pred = a.predict(X_test)


In [14]:
print("Accuracy: ", (np.sum(y_pred == y_test))/len(y_pred)*100,"%")

Accuracy:  70.0 %


9. (*) Test the modularity of the implementation applying it on a different dataset. Ideally, you shouldnot change the code of your KNN python class.
    - Download the MNIST dataset and sample only 100 points per digit. You will end up with adataset of 1000 samples. 
    - Define again four numpy arrays as you did in Exercises 2 and 3.
    - Apply your KNN as you did for the Iris dataset.
    - Evaluate the accuracy on MNIST’s y_test.

In [15]:
df_MNIST = pd.read_csv('../Datasets/mnist_test.csv', header=None)
df_MNIST

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# get just 100 points for digit

df_MNIST_edited = df_MNIST[df_MNIST[0] == 0][:100]
for i in range(1,10):
    df_MNIST_edited = df_MNIST_edited.append(df_MNIST[df_MNIST[0] == i][:100])
len(df_MNIST_edited)

1000

In [17]:
X_test_2 = df_MNIST_edited.sample(int(len(df_MNIST_edited)/5)) #20% means len/5
y_test_2 = X_test_2[0] # pandas' columns are already numpy arrays
X_test_2 = X_test_2.drop(columns=[0])

X_train_2 = df_MNIST_edited.drop(X_test_2.index)
X_train_2 = X_train_2.drop(columns=[0])
y_train_2  = df_MNIST_edited[0].drop(y_test_2.index)

In [18]:
mnist_knn = KNearestNeighbors(5,"euclidean","distance")
mnist_knn.fit(X_train_2,y_train_2)
y_pred_2 = mnist_knn.predict(X_test_2)

print("Accuracy: ",(np.sum(y_pred_2 == y_test_2))/len(y_pred_2)*100,"%")

Accuracy:  84.5 %


10. (*) The choice of parameters like k,distance_metric or weightsis part of your data analy sispipeline: the process is typically known as validation. You will learn more about algorithm validationin theory lectures.For now, evaluate the accuracy scores achieved by your KNN with different parameter values. Then,try to identify which is the best configuration, for both Iris and MNIST.

In [22]:
k_values = [3,5]
metrics = ["euclidean","manhattan","cosine"]
weights = ["uniform","distance"]

accuracy_MNIST = []
accuracy_IRIS = []

for k in k_values:
    for metric in metrics:
        for weight in weights:
            # IRIS 
            obj = KNearestNeighbors(k,metric,weight)
            obj.fit(X_train,y_train)
            y_pred = obj.predict(X_test)
            acc = (np.sum(y_pred == y_test))/len(y_pred)*100
            accuracy_IRIS.append([k,metric,weight,acc])
            
            # MNIST
            obj = KNearestNeighbors(k,metric,weight)
            obj.fit(X_train_2,y_train_2)
            y_pred_2 = obj.predict(X_test_2)
            acc_2 = (np.sum(y_pred_2 == y_test_2))/len(y_pred_2)*100
            accuracy_MNIST.append([k,metric,weight,acc_2]) 

In [23]:
accuracy_IRIS

[[3, 'euclidean', 'uniform', 100.0],
 [3, 'euclidean', 'distance', 100.0],
 [3, 'manhattan', 'uniform', 100.0],
 [3, 'manhattan', 'distance', 100.0],
 [3, 'cosine', 'uniform', 100.0],
 [3, 'cosine', 'distance', 100.0],
 [5, 'euclidean', 'uniform', 100.0],
 [5, 'euclidean', 'distance', 100.0],
 [5, 'manhattan', 'uniform', 100.0],
 [5, 'manhattan', 'distance', 100.0],
 [5, 'cosine', 'uniform', 100.0],
 [5, 'cosine', 'distance', 100.0]]

In [24]:
accuracy_MNIST

[[3, 'euclidean', 'uniform', 87.0],
 [3, 'euclidean', 'distance', 87.0],
 [3, 'manhattan', 'uniform', 85.5],
 [3, 'manhattan', 'distance', 85.5],
 [3, 'cosine', 'uniform', 89.0],
 [3, 'cosine', 'distance', 89.0],
 [5, 'euclidean', 'uniform', 84.5],
 [5, 'euclidean', 'distance', 84.5],
 [5, 'manhattan', 'uniform', 82.0],
 [5, 'manhattan', 'distance', 82.0],
 [5, 'cosine', 'uniform', 89.5],
 [5, 'cosine', 'distance', 89.5]]